In [1]:
!pip install psycopg2

     |████████████████████████████████| 383 kB 422 kB/s 
  Created wheel for psycopg2: filename=psycopg2-2.8.6-cp37-cp37m-linux_x86_64.whl size=452754 sha256=21f2f069bc4d1c5e18f13100ec5013c8a0733a4c05fd1cfe8bf8cb8a1561133c
  Stored in directory: /root/.cache/pip/wheels/25/78/75/9c0323f7e1fb42143cbd2439302beb7850a1034abb961cb281
Successfully built psycopg2


In [2]:
import numpy as np
import pandas as pd
import psycopg2


In [3]:
conn = psycopg2.connect("dbname=cricket host =sportsanalytics.in user=postgres password=Sumit@123 ")
series = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4 AND start_date BETWEEN '01-01-2008 00:00:01' AND '12-12-2020 00:00:01'", conn)
query = f"""SELECT id FROM cds_match_match WHERE series_id in {tuple(series['id'])} """
match_ids = pd.read_sql_query(query,conn)
query = f"""SELECT id,match_id FROM cds_scorecard_scorecard WHERE match_id in {tuple(match_ids['id'])}"""
scorecard_data = pd.read_sql_query(query,conn)
query = f"""SELECT * FROM cds_match_inning_matchbatsmaninning WHERE scorecard_id in {tuple(scorecard_data['id'])}"""
batting_data = pd.read_sql_query(query,conn)
query = f"""SELECT * FROM cds_match_inning_matchbowlersinning WHERE scorecard_id in {tuple(scorecard_data['id'])}"""
bowling_data = pd.read_sql_query(query,conn)

In [4]:
scorecard_data.columns = ['scorecard_id', 'match_id']
batting_data = batting_data.merge(scorecard_data,on = 'scorecard_id')
bowling_data = bowling_data.merge(scorecard_data,on = 'scorecard_id')

In [5]:
batting_data['is_out'] = (batting_data['how_out']!='not out')
batting_data = batting_data[['runs', 'balls', 'player_id','match_id','is_out']]

In [6]:
bowling_data['overs'] = bowling_data['overs'].astype(int)
bowling_data= bowling_data[['run_conceded','wickets','overs','player_id','match_id']]

In [7]:
batting_data = batting_data.sort_values(['match_id'])
bowling_data = bowling_data.sort_values(['match_id'])

In [8]:
batting_data.head()

,runs,balls,player_id,match_id,is_out
0,29,17,245,826,True
17,2,2,34,826,True
16,5,5,26,826,False
15,15,15,434,826,True
14,2,8,51,826,True


In [9]:
bowling_data.head()

,run_conceded,wickets,overs,player_id,match_id
0,26,3,4,609,826
11,3,0,1,30,826
10,14,1,4,677,826
9,14,2,4,22,826
7,39,1,3,674,826


In [10]:
batsman_ids = np.unique(np.array(batting_data['player_id']))
bowler_ids = np.unique(np.array(bowling_data['player_id']))

In [11]:
batsman_average = np.zeros([1650,2850])
batsman_strike_rate = np.zeros([1650,2850])
bowler_average = np.zeros([1650,2850])
bowler_strike_rate = np.zeros([1650,2850])
bowler_economy = np.zeros([1650,2850])

In [12]:
match_ids = np.unique(np.array(batting_data['match_id']))
a = 0
for player_id in batsman_ids:
    a = a+1
    print(a)
    runs = 0
    balls =0
    outs = 0
    for match_id in match_ids:
        if(batting_data[np.array(batting_data['match_id']==match_id)*np.array(batting_data['player_id']==player_id)].shape[0]):
            runs = runs + int(batting_data[np.array(batting_data['match_id']==match_id)*np.array(batting_data['player_id']==player_id)]['runs'])
            balls = balls + int(batting_data[np.array(batting_data['match_id']==match_id)*np.array(batting_data['player_id']==player_id)]['balls'])
            outs = outs + int(batting_data[np.array(batting_data['match_id']==match_id)*np.array(batting_data['player_id']==player_id)]['is_out'])
            if(balls):
                batsman_strike_rate[player_id][match_id] = round(100*runs/balls,2)
            if(outs==0):
                batsman_average[player_id][match_id] = runs
            if(outs):
                batsman_average[player_id][match_id] = round(runs/outs,2)
        else:
            if(balls):
                batsman_strike_rate[player_id][match_id] = round(100*runs/balls,2)
            if(outs==0):
                batsman_average[player_id][match_id] = runs
            if(outs):
                batsman_average[player_id][match_id] = round(runs/outs,2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
match_ids = np.unique(np.array(batting_data['match_id']))
a = 0
for player_id in bowler_ids:
    a = a+1
    print(a)
    runs = 0
    balls =0
    wickets = 0
    for match_id in match_ids:
        if(bowling_data[np.array(bowling_data['match_id']==match_id)*np.array(bowling_data['player_id']==player_id)].shape[0]):
            runs = runs + int(bowling_data[np.array(bowling_data['match_id']==match_id)*np.array(bowling_data['player_id']==player_id)]['run_conceded'])
            balls = balls + 6*int(bowling_data[np.array(bowling_data['match_id']==match_id)*np.array(bowling_data['player_id']==player_id)]['overs'])
            wickets = wickets + int(bowling_data[np.array(bowling_data['match_id']==match_id)*np.array(bowling_data['player_id']==player_id)]['wickets'])
            if(wickets):
                bowler_strike_rate[player_id][match_id] = round(balls/wickets,2)
                bowler_average[player_id][match_id] = round(runs/wickets,2)
            if(balls):
                bowler_economy[player_id][match_id] = round(runs*6/balls,2)
                
        else:
            if(wickets):
                bowler_strike_rate[player_id][match_id] = round(balls/wickets,2)
                bowler_average[player_id][match_id] = round(runs/wickets,2)
            if(balls):
                bowler_economy[player_id][match_id] = round(runs*6/balls,2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
for player_id in batsman_ids:
    for i in range(1,len(batsman_average[player_id])):
        if(batsman_average[player_id][i]==0):
            batsman_average[player_id][i] = batsman_average[player_id][i-1]
        if(batsman_strike_rate[player_id][i]==0):
            batsman_strike_rate[player_id][i] = batsman_strike_rate[player_id][i-1]
            
        

In [15]:
for player_id in batsman_ids:
    for i in range(1,len(bowler_average[player_id])):
        if(bowler_economy[player_id][i]==0):
            bowler_economy[player_id][i] = bowler_economy[player_id][i-1]
        if(bowler_average[player_id][i]==0):
            bowler_average[player_id][i] =  bowler_average[player_id][i-1]
        if(bowler_strike_rate[player_id][i]==0):
            bowler_strike_rate[player_id][i] = bowler_strike_rate[player_id][i-1]
        
        

In [16]:
np.savetxt('batsman_strike_rate.csv',batsman_strike_rate, delimiter=',',fmt='%1.3f')
np.savetxt('batsman_average.csv',batsman_average, delimiter=',',fmt='%1.3f')
np.savetxt('bowler_strike_rate.csv',bowler_strike_rate, delimiter=',',fmt='%1.3f')
np.savetxt('bowler_average.csv',bowler_average, delimiter=',',fmt='%1.3f')
np.savetxt('bowler_economy.csv',bowler_economy, delimiter=',',fmt='%1.3f')